In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
import scipy.io
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16,ResNet50
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape,LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal

<H1>DATA PREPARATION</H1>

In [ ]:
BATCH_SIZE=32
INPUT_DIM=224
NUM_EPOCHS=20

In [ ]:
train_directory='...'

In [ ]:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    validation_split=0.1,
    subset='training',
    seed=999,
    image_size=(INPUT_DIM,INPUT_DIM),
    batch_size=BATCH_SIZE,)
val_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    validation_split=0.1,
    subset='validation',
    seed=999,
    image_size=(INPUT_DIM,INPUT_DIM),
    batch_size=BATCH_SIZE,)

In [ ]:
class_names=train_dataset.class_names

In [ ]:
plt.figure(figsize=(20,10))

In [ ]:
for images,labels in train_dataset.take(1):
    for i in range(BATCH_SIZE):
        ax=plt.subplot(BATCH_SIZE//8,BATCH_SIZE//4,i+1)
        plt.imshow(images[i]/255.)
        plt.title(class_names[labels[i]])

In [ ]:
train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
val_dataset=val_dataset.prefetch(buffer_size=AUTOTUNE)

<H1>MODELING</H1>

In [ ]:
backbone = ResNet50(
        weights='imagenet',
        input_shape=(INPUT_DIM,INPUT_DIM,3),
        include_top=False,)
backbone.trainable=False
model=Sequential([
    Input(shape=(224,224,3)),
    backbone,
    GlobalAveragePooling2D(),
    Dense(1,activation='sigmoid'),
])

In [ ]:
model.summary()

<H1>TRAINING</H1>

In [ ]:
bce_loss=BinaryCrossentropy()
optimizer=Adam(lr=1e-3)

train_accuracy=BinaryAccuracy()
val_accuracy=BinaryAccuracy()

train_tp=TruePositives()
val_tp=TruePositives()

train_tn=TrueNegatives()
val_tn=TrueNegatives()

train_fp=FalsePositives()
val_fp=FalsePositives()

train_fn=FalseNegatives()
val_fn=FalseNegatives()

train_precision=Precision()
val_precision=Precision()

train_recall=Recall()
val_recall=Recall()

In [ ]:
@tf.function
def train_step(x,y):
    with tf.GradientTape() as tape:
        y_pred=model(x,training=True)
        loss_value=bce_loss(y,y_pred)
    gradients=tape.gradient(loss_value,model.trainable_weights)
    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    
    train_accuracy.update_state(y,y_pred)
    train_tp.update_state(y,y_pred)
    train_tn.update_state(y,y_pred)
    train_fp.update_state(y,y_pred)
    train_fn.update_state(y,y_pred)
    train_precision.update_state(y,y_pred)
    train_recall.update_state(y,y_pred)
    
    return loss_value

In [ ]:
@tf.function
def val_step(x,y):
    y_pred=model(x,training=False)
    loss_value=bce_loss(y,y_pred)
    val_accuracy.update_state(y,y_pred)
    val_tp.update_state(y,y_pred)
    val_tn.update_state(y,y_pred)
    val_fp.update_state(y,y_pred)
    val_fn.update_state(y,y_pred)
    val_precision.update_state(y,y_pred)
    val_recall.update_state(y,y_pred)
    
    return loss_value

In [ ]:
for epoch in range(NUM_EPOCHS):
    for (x_train,y_train) in train_dataset:
        train_loss=train_step(x_train,y_train)
    for (x_val,y_val) in val_dataset:
        val_loss=val_step(x_val,y_val)
    template='Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}\n '
    print(template.format(epoch+1,
                         train_loss,train_accuracy.result()*100,train_precision.result()*100,train_recall.result()*100))
    
    print('-------------------------------------------------------------------------------------')
    print('            TP: {} --------------            FP:{}         --------------------'.format(train_tp.result(),train_fp.result()))
    print('-------------------------------------------------------------------------------------')
    print('            FN: {} --------------            TN:{}         --------------------'.format(train_fn.result(),train_tn.result()))
    print('-------------------------------------------------------------------------------------')

    template='Validation Loss: {:.2f}, Validation Accuracy: {:.2f}, Validation Precision: {:.2f}, Validation Recall: {:.2f}\n '
    print(template.format(val_loss,val_accuracy.result()*100,val_precision.result()*100,val_recall.result()*100))
    
    print('-------------------------------------------------------------------------------------')
    print('            Val TP: {} --------------          Val FP:{}         --------------------'.format(val_tp.result(),val_fp.result()))
    print('-------------------------------------------------------------------------------------')
    print('            Val FN: {} --------------          Val TN:{}         --------------------'.format(val_fn.result(),val_tn.result()))
    print('-------------------------------------------------------------------------------------')
    
    train_accuracy.reset_states()
    train_tp.reset_states()
    train_tn.reset_states()
    train_fp.reset_states()
    train_fn.reset_states()
    train_precision.reset_states()
    train_recall.reset_states()
    
    val_accuracy.reset_states()
    val_tp.reset_states()
    val_tn.reset_states()
    val_fp.reset_states()
    val_fn.reset_states()
    val_precision.reset_states()
    val_recall.reset_states()
        

In [ ]:
y_val=[]
x_val=[]

for x,y in val_dataset:
    x_val.append(x)
    y_val.append(y)

In [ ]:
y_true=tf.stack(y_val,axis=0)[0]
y_pred=model.predict(tf.stack(x_val,axis=0)[0])

In [ ]:
def roc(labels,predictions):
    fp,tp,_=sklearn.metrics.roc_curve(labels,predictions)
    plt.plot(fp,tp)
    plt.xlabel('false positives rate')
    plt.ylabel('true positives rate')
    plt.grid(True)

In [ ]:
roc(y_true,y_pred[...,0])

<h1>TESTING</h1>

In [ ]:
im_array=img_to_array(load_img(image,target_size=(INPUT_DIM,INPUT_DIM)))
if mode.prdict(tf.expand_dims(im_array,axis=0))[0][0]<0.5:
    print('its benign')
else:
    print('malignant')